In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Input, Convolution2D, Activation, concatenate, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from tensorflow.keras.utils  import get_source_inputs, get_file
from keras import backend as K
from keras_applications.imagenet_utils import _obtain_input_shape

In [2]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Modular function for Fire Node

In [3]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

In [4]:
# Original SqueezeNet from paper.

def SqueezeNet(include_top=True, weights='imagenet',
               input_tensor=None, input_shape=None,
               pooling=None,
               classes=1000):
    """Instantiates the SqueezeNet architecture.
    """
        
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')


    input_shape = _obtain_input_shape(input_shape,
                                      default_size=200,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor


    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    if include_top:
        # It's not obvious where to cut the network... 
        # Could do the 8th or 9th layer... some work recommends cutting earlier layers.
    
        x = Dropout(0.5, name='drop9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = Activation('relu', name='relu_conv10')(x)
        x = GlobalAveragePooling2D()(x)
        x = Activation('softmax', name='loss')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling=='max':
            x = GlobalMaxPooling2D()(x)
        elif pooling==None:
            pass
        else:
            raise ValueError("Unknown argument for 'pooling'=" + pooling)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='squeezenet')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
            
        model.load_weights(weights_path)



    return model

In [5]:
model = SqueezeNet(include_top=True)

model.summary()


Model: "squeezenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 99, 99, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 relu_conv1 (Activation)        (None, 99, 99, 64)   0           ['conv1[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 49, 49, 64)   0           ['relu_conv1[0][0]']    

In [15]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dense, Flatten, Input, GlobalAveragePooling2D, Convolution2D, Dropout

NUM_OF_CLASSES = 29
pixels = 200
IMAGE_SIZE = (pixels, pixels)
x = model.layers[-6].output
x = Dropout(0.5, name='drop9')(x)
ConvLayer = Convolution2D(NUM_OF_CLASSES, (1, 1), padding='valid', name='conv10')(x)
ConvLayer = Activation('relu', name='relu_conv10')(ConvLayer)
Flatten_Layer = GlobalAveragePooling2D()(ConvLayer)
output = Dense(units= NUM_OF_CLASSES, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = 'softmax')(Flatten_Layer)
model = Model(inputs = model.input, outputs= output)

for layer in model.layers[:-10] :
    layer.trainable = False
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 99, 99, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 relu_conv1 (Activation)        (None, 99, 99, 64)   0           ['conv1[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 49, 49, 64)   0           ['relu_conv1[0][0]']       

In [2]:
pixels = 200
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])


data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()   #hna a3tkd kan mafroud n7ot gwa el repeat el rkm el epoch kol el sites bt3ml kda

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Input size (200, 200)
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [21]:
model.compile(optimizer= Adam(learning_rate= 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch = train_size  // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

model.fit(x = train_ds, validation_data=val_ds, epochs=5, verbose=1, steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps)

Epoch 1/5
4350/4350 [==============================] - 854s 196ms/step - loss: 0.5798 - accuracy: 0.8370 - val_loss: 0.5545 - val_accuracy: 0.8460
Epoch 2/5
4350/4350 [==============================] - 140s 32ms/step - loss: 0.5425 - accuracy: 0.8480 - val_loss: 0.5206 - val_accuracy: 0.8567
Epoch 3/5
4350/4350 [==============================] - 139s 32ms/step - loss: 0.5110 - accuracy: 0.8563 - val_loss: 0.4870 - val_accuracy: 0.8671
Epoch 4/5
4350/4350 [==============================] - 142s 33ms/step - loss: 0.4824 - accuracy: 0.8646 - val_loss: 0.4564 - val_accuracy: 0.8772
Epoch 5/5
4350/4350 [==============================] - 141s 32ms/step - loss: 0.4570 - accuracy: 0.8717 - val_loss: 0.4367 - val_accuracy: 0.8818


In [22]:
model.save("SqueezeNet-Test-87.2%.h5")

In [23]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
model = tf.keras.models.load_model('SqueezeNet-Test-87.2%.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [24]:
saved_model_path = "Saved_Model_With_TF1"
tf.saved_model.save(model, saved_model_path)
model.save("SqueezeNet-Test.h5")
print("Saved Model")
saved_model_path = "Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(model, saved_model_path)

INFO:tensorflow:Assets written to: Saved_Model_With_TF1\assets


INFO:tensorflow:Assets written to: Saved_Model_With_TF1\assets


Saved Model


INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras\assets


INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras\assets
